# Root Logger

The root logger is the top-level log interface in the application.  By default, it has no handlers or filters. Because it has no handlers, it has "last resort" behavior - it prints to `stderr` only messages with level `WARNING` or above.  You can __raise the level__ to make it more strict, but you cannot make it less strict because the "last resort behavior" will filter out anything below `WARNING` anyway.

The levels (which have corresponding methods), from lowest to highest are:
  - DEBUG
  - INFO
  - WARNING
  - ERROR
  - CRITICAL
  
`NOTSET` is a special pass-through value.

You can either call the specific methods like `logger.critical()` or call `logger.log()` with the level as an argument, as shown below.

__WARNING__: you must __restart the kernel__ before running this cell if you ran any others (including this one).

In [1]:
import logging

logger = logging.getLogger() # root logger because no name argument
logger.info('hello') # discarded because lower than WARNING
logger.warning('hi') # printed to stderr
logger.log(logging.WARNING, 'hi again') # printed to stderr

print('handlers:', logger.handlers)
print('filters:', logger.filters)
print('effective level:', logger.getEffectiveLevel()) # defaults to WARNING
print('logger level:', logger.level) # defaults to WARNING
print()

print('DEBUG level:', logging.DEBUG)
print('INFO level:', logging.INFO)
print('WARNING level:', logging.WARNING)
print('ERROR level:', logging.ERROR)
print('CRITICAL level:', logging.CRITICAL)
print('NOTSET level:', logging.NOTSET)
print()

logger.setLevel(logging.INFO)
logger.info('hi hi hi') # dropped because of default last resort handler level
logger.setLevel(logging.CRITICAL)
logger.warning('yo yo yo') # dropped because logger is more restrict than (lack of) handler

hi
hi again


handlers: []
filters: []
effective level: 30
logger level: 30

DEBUG level: 10
INFO level: 20
WARNING level: 30
ERROR level: 40
CRITICAL level: 50
NOTSET level: 0



# Default Stream Handler

Adding the default `StreamHandler` to the global logger makes it still behave the same, but now you have a point of configurability - you can change the properties of the handler.

By default, `StreamHandler` has a level of `NOTSET` and prints to `stderr`.  `NOTSET` makes the handler defer to the logger for its level, which means just by adding a default stream handler, you can now raise and lower the logger's level at will!

When the __handler's level is set__, as shown below, then it will filter in addition to the logger's level (__highest one wins__).

__WARNING__: you must __restart the kernel__ before running this cell if you ran any others (including this one).

In [1]:
import logging

def print_data(logger, prefix):
    logger.info(prefix + ' info')
    logger.warning(prefix + ' warning')
    print('logger level:', logger.level)
    print('handler level:', logger.handlers[0].level)
    print('logger effective level:', logger.getEffectiveLevel())
    
logger = logging.getLogger()
logger.addHandler(logging.StreamHandler())
print_data(logger, 'default')

logger.setLevel(logging.DEBUG)
print_data(logger, 'logger-level') # everything already gets through

logger.handlers[0].setLevel(logging.DEBUG)
print_data(logger, 'handler-level') # no change (everything still through)

logger.handlers[0].setLevel(logging.WARNING)
print_data(logger, 'handler-high-level')

default warning
logger-level info
logger-level warning
handler-level info
handler-level warning
handler-high-level warning


logger level: 30
handler level: 0
logger effective level: 30
logger level: 10
handler level: 0
logger effective level: 10
logger level: 10
handler level: 10
logger effective level: 10
logger level: 10
handler level: 30
logger effective level: 10


# Global Singleton and Multiple Handlers

The root logger is a __global singleton__ - every single call to `getLogger()` will get the same instance back, allowing you to use and configure it anywhere in the appliation (from __any module__).

When multiple handlers are present, messages go to __all handlers in parallel__ with their own level filtering.

__NOTE__: loggers are global singletons, but things that you add to them like __handlers, formatters, and filters are not__.

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging

def print_data(prefix):
    logger = logging.getLogger()
    logger.debug(prefix + ' debug')
    logger.info(prefix + ' info')
    logger.warning(prefix + ' warning')
    
print_data('default')

logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().handlers[0].setLevel(logging.DEBUG)
logging.getLogger().handlers[1].setLevel(logging.INFO)
logging.getLogger().setLevel(logging.DEBUG)

print_data('handlers')

default warning
handlers debug
handlers info
handlers info
handlers warning
handlers warning


# Stream Handler to Stdout

`StreamHandler` can be created to go to `stdout` instead of the default `stderr`, or in fact __any stream__ you have a reference to.

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging, sys

logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(sys.stdout))

logger.warning('stdout!') # white background instead of red this time!

stdout!


# Other Handlers

- `FileHandler` for streaming to a file on disk
- `NullHandler` for throwing away messages
- `SocketHandler` for logging over the network
- `HTTPHandler` for logging to a REST server
- etc. etc. (see https://docs.python.org/3/library/logging.handlers.html)

# Named Loggers

Like the root logger, named loggers are __global singletons__.  They also form an implicit __hierarchy by module name__.  For instance, `my-logger.sub-logger` is understood by the `logging` module to be the child of `my-logger`, which is in turn the __child of the root logger__.

If the level of any logger in the hierarchy is `NOTSET`, then it will take its level, its `getEffectiveLevel()` value, from the first ancestor that has an explicit value.

By default, __all handlers in the hierarchy__ chain will see the message (depending on their levels if set or the levels of their loggers if not set). So, for instance, if you add global handlers and then add specific handlers, you'll get __duplicate logging__, which might be desired in some situations (eg. different destinations, formatters, etc.).

To prevent parent and root handlers from seeing log messages, you can set `logger.propagate` to `False`.  If the logger doesn't have handlers, you will get the "last resort" behavior again (`WARNING` to `stderr`).

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging, sys

# configure root logger to go to stdout AND stderr with DEBUG level
rootLogger = logging.getLogger()
rootLogger.addHandler(logging.StreamHandler(sys.stdout))
rootLogger.addHandler(logging.StreamHandler(sys.stderr))
rootLogger.setLevel(logging.DEBUG)

# configure named logger
logger = logging.getLogger('my-logger')

# log some stuff
logger.info('my-logger info')
logger.warning('my-logger warning')
print(logger.level)  # NOTSET
print(logger.getEffectiveLevel()) # DEBUG (from root)
print(logger.handlers) # no handlers (so goes to root)
print()

# configure lower-level named logger
logger = logging.getLogger('my-logger.sub-logger')

# log more stuff
logger.info('sub-logger info')
logger.warning('sub-logger warning')
print(logger.level)  # NOTSET
print(logger.getEffectiveLevel()) # DEBUG (from root)
print(logger.handlers) # no handlers (so goes to root)
print()

# intercept config in the parent (WARNING, stderr)
parent = logging.getLogger('my-logger') # same as above because singleton
parent.setLevel(logging.WARNING)
parent.addHandler(logging.StreamHandler(sys.stdout))

# duplicated logging
logger.info('sub2-logger info') # dropped because parent is DEBUG
logger.warning('sub2-logger warning') # 3 times because 3 handlers in hierarchy
print(logger.level)  # NOTSET
print(logger.getEffectiveLevel()) # WARNING (from parent)
print(logger.handlers) # no handlers (so goes to root)
print(parent.handlers)
print()

# non-duplicated logging
logger.propagate = False
logger.warning('sub3-logger warning')
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.warning('sub4-logger warning')

my-logger info


my-logger info


my-logger warning


my-logger warning


0
10
[]

sub-logger info


...

sub-logger warning


sub-logger warning


0
10
[]

sub2-logger warning
sub2-logger warning


sub2-logger warning
sub3-logger warning


0
30
[]
[<StreamHandler stdout (NOTSET)>]

sub4-logger warning


# Naming Convention

By convention, loggers are usually created like this at the top of modules: `logger = logging.getLogger(__name__)`

This makes sure that each module (or the top-level script) has its own logger and that the logger hierarchy mirrors the __module hierarchy__.

In a __top-level script__, you might instead choose to use the root logger or the name of the app.

# Top-Down Configuration

Given the above shadowing behavior and naming convention, you would typically do the following:
1. Set the __root logger__ for base configuration.
1. Override as necessary at each level for more specific behavior.
1. Only change log settings at the tops of modules to keep things predictable.

# Formatters

Formatters are instances of `logging.Formatter` based on __format strings__ that recognize certain built-in specifiers (https://docs.python.org/3/library/logging.html#logrecord-attributes).

A __handler__ has a __single formatter__ which defaults to just printing the message if not set. There is __no propagation__ of formatters - they belong to the handlers they are set on only. They also don't have their own levels.

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging, sys

# configure root logger to go to stdout AND stderr with DEBUG level
rootLogger = logging.getLogger()
rootLogger.addHandler(logging.StreamHandler(sys.stdout))
rootLogger.addHandler(logging.StreamHandler(sys.stderr))
rootLogger.setLevel(logging.DEBUG)

# configure named logger
logger = logging.getLogger('my-logger')
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.handlers[0].setFormatter(logging.Formatter('%(levelname)s - %(asctime)s: %(message)s'))

# log some stuff
logger.info('my-logger info')
logger.warning('my-logger warning')

INFO - 2025-08-12 16:30:53,493: my-logger info
my-logger info


my-logger info


WARNING - 2025-08-12 16:30:53,495: my-logger warning
my-logger warning


my-logger warning


# Logging Format Strings

Logging methods can take __old-style format strings__ and additional args for the format args. The reason this is done is to make sure that string interpolation isn't done until necessary as an __optimization__. Use this instead of doing your own interpolation.

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging

logger = logging.getLogger()
logger.warning('%s %s', 'cat', 'dog')
logger.log(logging.WARNING, '%s %s', 'apple', 'banana')

cat dog
apple banana


# Logging Exceptions

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging

logger = logging.getLogger()

# not in an exception context
logger.exception('uh ohs!')
logger.warning('')

#exception context
try:
    1 / 0
except:
    logger.exception('uh ohs again!')

uh ohs!
NoneType: None

uh ohs again!
Traceback (most recent call last):
  File "/var/folders/wn/pmx5pn155tg83bskqqc2wzz00000gn/T/ipykernel_66887/2797187029.py", line 11, in <module>
    1 / 0
ZeroDivisionError: division by zero


# basicConfig()

`logging.basicConfig()` is a helper function to configure your root logger and its handler(s), formatter(s) etc. based on named arguments.

You would typically call it once at the beginning of the app to set the default behavior and then get your logger instances in modules and override anything that's necessary for them (or just use them).

Note that it will create a handler on the root logger (just a default `StreamHandler`) even if you don't set a handler or formatter.  So even just calling this to set the level is already more convenient.

Other options include passing a __list of handlers__, passing a __file to mirror__ log messages to, etc.

__WARNING__: you must restart the kernel before running this cell if you ran any others (including this one).

In [1]:
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

logger = logging.getLogger('my_logger')
logger.info('hello')
print(logging.getLogger().handlers)

2025-08-12 16:47:05,134 - my_logger - INFO - hello


[<StreamHandler stderr (NOTSET)>]
